In [1]:
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
from random import sample
import numpy as np
from sklearn import svm
from sklearn import preprocessing
from sklearn import metrics
from sklearn import cross_validation
from sklearn import grid_search

In [2]:
# SVM CLASSIFICATION FUNCTIONS
def randomSplit(X,y):
    accuracy = np.array([])
    # do 20 random splits
    for i in range(20):
        # leave 20% out for testing
        skf = cross_validation.StratifiedKFold(y,n_folds=5,shuffle=True)
    
    for cv_i,test_i in skf:
        train_X = X[cv_i]
        train_y = y[cv_i]
        # do training here
        clf = gridSearchSVM(train_X,train_y)

    # do testing here
    test_X = X[test_i]
    test_y = y[test_i]
    prediction = clf.predict(test_X)
    # record performance
    foo = performance(prediction, test_y)
    accuracy = np.append(accuracy, foo)
    mean_acc = np.mean(accuracy)
    return mean_acc


def gridSearchSVM(X,y):
    parameters = {'kernel':('linear','rbf'), 'C':[1,20]}
    svr  = svm.SVC()
    clf = grid_search.GridSearchCV(svr, parameters, cv = 5, n_jobs = 4, verbose=1)
    clf.fit(X,y)
    return clf

def performance(prediction, target):
    acc = metrics.accuracy_score(target, prediction, normalize=True)
    return acc

In [3]:
# DATA LOADING FUNCTIONS
def loadYaleData():
    X = np.matrix(scipy.io.loadmat('yalefacesFFT.mat')['DATA'])
    individuals = 15; 
    picsPerInd = 11;
    Y = np.zeros((individuals*picsPerInd))
    for i in range(0,individuals):
        Y[(i)*picsPerInd:(i+1)*picsPerInd] = i;
    return (X,Y)

def loadOlivettiData():
    X = np.matrix(scipy.io.loadmat('olivettifacesFFT.mat')['DATA'])
    individuals = 40; 
    picsPerInd = 10;
    Y = np.zeros((individuals*picsPerInd))
    for i in range(0,individuals):
        Y[(i)*picsPerInd:(i+1)*picsPerInd] = i;
    return (X,Y)

In [4]:
# TRANFORMATION FUNCTIONS

def removePhase(V):
    return V[0:V.shape[0]/2].reshape(V.shape[0]/2);

In [8]:
# RECONSTRUCTION FUNCTIONS

# Reconstruct an image from the Yale database 
# based on the all frequecies and phases
def getYaleface(V):
    R = V[0:319*122];
    theta = V[319*122:];

    # set reshape to FORTRAN order since that is what matlab uses
    R = np.reshape(R, (122,319),order='F')
    theta = np.reshape(theta, (122,319),order='F')

    FFTimg = np.multiply(R,np.exp(np.sqrt(-1+0j)*theta));
    FFTimg = np.concatenate((FFTimg, np.conj(np.rot90(FFTimg[0:121,:],2))), axis=0);       
    img = np.fft.ifft2(np.fft.ifftshift(FFTimg));
    
    return np.real(img)

# Reconstruct an image from the Olivetti database 
# based on the all frequecies and phases
def getOlivettiface(V):
    R = V[0:63*32];
    theta = V[63*32:];

    # set reshape to FORTRAN order since that is what matlab uses
    R = np.reshape(R, (32,63),order='F')
    theta = np.reshape(theta, (32,63),order='F')

    FFTimg = np.multiply(R,np.exp(np.sqrt(-1+0j)*theta));
    FFTimg = np.concatenate((FFTimg, np.conj(np.rot90(FFTimg[0:31,:],2))), axis=0);       
    img = np.fft.ifft2(np.fft.ifftshift(FFTimg));
    
    return np.real(img)

def addZeroPhase(V,i, X):
    F = np.zeros((V.shape[0]*2))
    print X[V.shape[0]:, i].shape
    #X[0:V.shape[0]] = V[0:V.shape[0]].reshape(V.shape[0]);
    F[0:V.shape[0]] = V[:].reshape(V.shape[0])
    F[0:V.shape[0]] = X[V.shape[0]:, i].reshape(V.shape[0]);
    
    return F.reshape(F.shape[0]);

In [6]:
# DISPLAY FUNCTIONS
# h,w are height and width of the individual images, nh,nw are the dimensions of the 
# collage interms of images 
def makeCollage(h, w, nh, nw, DATA, getImage):
    border = 10
    collage = np.zeros((h*nh+border*(nh+1),w*nw+border*(nw+1)))
    randImg = sample(xrange(DATA.shape[1]), nw*nh)
    
    for i in range(0,nw):
        for j in range(0,nh):
            img = getImage(DATA[:,randImg[nw*i + j]])
            collage[(i+1)*border-1+i*h:(i+1)*h+(i+1)*border-1, (j+1)*border+j*w-1:(j+1)*border-1+(j+1)*w] = img

    plt.imshow(collage);
    plt.gray()
    plt.show()
    

In [ ]:
(X,Y) = loadYaleData()
makeCollage(243, 319, 10, 10, X, getYaleface)
(X,Y) = loadOlivettiData()
makeCollage(63, 63, 10, 10, X, getOlivettiface)

In [ ]:
yaleHeight = 243
yaleWidth = 319
olivettHeight = 63
olivettWidth = 63

(X,Y) = loadYaleData()


xNoPhase = np.zeros([X.shape[0]/2, X.shape[1]])

for i in range(0,X.shape[1]):
    xNoPhase[:,i] = removePhase(X[:,i])[:]

xZeroPhase = np.zeros([X.shape[0], X.shape[1]])
for i in range(0,X.shape[1]):
    xZeroPhase[:,i] = addZeroPhase(xNoPhase[:,i],i,X)[:]

print X.shape
print xNoPhase.shape
print xZeroPhase.shape

makeCollage(yaleHeight, yaleWidth, 2, 2, xZeroPhase, getYaleface)

print randomSplit(np.transpose(xNoPhase),Y)

(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)
(38918L, 1L)